# Block B Starter / Executor (Colab)
Use this notebook to:
1) Set up the repo for Block B
2) Configure dataset paths (Cityscapes, nuScenes-mini, CMP)
3) Run a **dry test** of the new data pipeline on CMP Facade

**Note:** End-to-end model inference is wired via the existing CLI (`stg-stsg-model/src/infer_v1.py`).
At this stage, features are dummy-cached; we patch the runner to call the CLI per scene.

In [ ]:
#@title Verify Colab path
import os
assert os.path.isdir('/content/stg-system-main'), "❌ Repo not found at /content/stg-system-main. Re-clone before running."
print('✅ Repo path verified. You can now run all cells top-to-bottom.')


In [ ]:
#@title 0) Environment check (GPU & Python)
import sys, platform, subprocess, os
print("Python:", sys.version)
print("Platform:", platform.platform())
!nvidia-smi || echo "No GPU visible (CPU-only run)."


## 1) Bring the code into Colab
We now clone the public repository directly from GitHub instead of uploading a ZIP.

In [ ]:
#@title Clone the repo
REPO_URL = "https://github.com/azsanche-asg/stg-system.git"  # public repo URL
EXTRACT_DIR = "/content/stg-system-main"

import os, shutil

# Remove any previous clone to ensure a clean copy
shutil.rmtree(EXTRACT_DIR, ignore_errors=True)

print(f"Cloning from {REPO_URL} ...")
!git clone "$REPO_URL" "$EXTRACT_DIR"

print("✅ Cloned to:", EXTRACT_DIR)
!find /content/stg-system-main -maxdepth 2 -type d -print


## 2) Install minimal requirements
This keeps installs light; heavy backbones will be added later.

In [ ]:
#@title Install project requirements (lightweight)
%pip -q install -U pip
%pip -q install numpy pyyaml tqdm opencv-python matplotlib pillow torch torchvision


In [ ]:
#@title Install feature extractors (CLIP/MiDaS/DINO/SAM)%pip -q install ftfy regex tqdm%pip -q install git+https://github.com/openai/CLIP.gitimport torch, osprint("Torch:", torch.__version__, "CUDA:", torch.cuda.is_available())# Allow git-based torch.hub downloads in Colabos.environ["TORCH_HOME"] = "/content/torch_hub"os.makedirs(os.environ["TORCH_HOME"], exist_ok=True)

## 3) Quick repo sanity & path setup

In [ ]:
#@title Add repo to sys.path
import sys, os
ROOT = "/content/stg-system-main/stg-system-main"
assert os.path.isdir(ROOT), "Repo root not found."
sys.path.append(ROOT)
print("Repo root:", ROOT)

# Show key modules
!ls -la "$ROOT"
!ls -la "$ROOT/stg-real-eval" || echo "stg-real-eval not found (ensure you applied the Block B patch)."


## 4) Configure dataset paths
Fill in your local paths. For a **dry test**, you can point CMP to a folder with 5–10 JPGs.

In [ ]:
#@title Set dataset paths (edit these)
CITYSCAPES_SEQ_ROOT = "/content/cityscapes/leftImg8bit_sequence/demoVideo"  #@param {type:"string"}
CITYSCAPES_STILL_ROOT = "/content/cityscapes/leftImg8bit/val"               #@param {type:"string"}
CMP_ROOT = "/content/cmp/facade_db"                                         #@param {type:"string"}
NUSCENES_ROOT = "/content/nuscenes-mini"                                     #@param {type:"string"}

# Patch CMP config inline
import yaml, json
from pathlib import Path
cfg_path = Path(ROOT) / "stg-real-eval/configs/block_b_cmp.yaml"
cfg = yaml.safe_load(cfg_path.read_text())
cfg["paths"]["root"] = CMP_ROOT
(out_path:=cfg_path).write_text(yaml.safe_dump(cfg))
print("Wrote CMP path into:", out_path)
print(Path(out_path).read_text())


In [ ]:
#@title Verify dataset pathsimport osprint("CMP_ROOT:", CMP_ROOT, "exists:", os.path.isdir(CMP_ROOT))print("CITYSCAPES_SEQ_ROOT:", CITYSCAPES_SEQ_ROOT, "exists:", os.path.isdir(CITYSCAPES_SEQ_ROOT))print("CITYSCAPES_STILL_ROOT:", CITYSCAPES_STILL_ROOT, "exists:", os.path.isdir(CITYSCAPES_STILL_ROOT))print("NUSCENES_ROOT:", NUSCENES_ROOT, "exists:", os.path.isdir(NUSCENES_ROOT))

## 5) (Optional) Create a tiny debug CMP folder
If you don't have CMP yet, generate 6 dummy images to exercise the pipeline.

In [ ]:
#@title Create dummy CMP images (optional)
import os
from PIL import Image, ImageDraw
from pathlib import Path
import numpy as np

if not os.path.exists(CMP_ROOT):
    os.makedirs(CMP_ROOT, exist_ok=True)
    for i in range(6):
        img = Image.new("RGB", (640, 384), (220, 220, 230))
        d = ImageDraw.Draw(img)
        for x in range(40, 600, 60):
            for y in range(40, 320, 70):
                d.rectangle([x, y, x+30, y+20], outline=(0,0,0))
        img.save(f"{CMP_ROOT}/cmp_{i:03d}.jpg")
print("CMP_ROOT contains:", len(list(Path(CMP_ROOT).glob('*.jpg'))) + len(list(Path(CMP_ROOT).glob('*.png'))), "images")
!ls -1 "$CMP_ROOT" | head


## 7) **Dry test** on CMP
This invokes the end-to-end Block B runner with the CMP config.

In [ ]:
#@title Run dry test (CMP)
%cd $ROOT
!python stg-real-eval/src/run_eval_real.py --config stg-real-eval/configs/block_b_cmp.yaml
!echo "Results:"
!find results/block_b/cmp -maxdepth 1 -type f -name "*.json" -print


By default all four extractors run (CLIP, DINO, MiDaS, SAM).
To limit which ones are extracted during development, set the environment variable `STG_FEATURES` before the run, e.g.:
```python
import os
os.environ['STG_FEATURES'] = 'clip,dino'
```
If present, the scripts will only cache those features.

In [ ]:
#@title Run CMP with real feature cache (Phase 2A)%cd $ROOT# Ensure CMP config points to current pathimport yamlfrom pathlib import Pathcfg_path = Path("stg-real-eval/configs/block_b_cmp.yaml")cfg = yaml.safe_load(cfg_path.read_text())cfg["paths"]["root"] = CMP_ROOTcfg_path.write_text(yaml.safe_dump(cfg))# Run evaluation (this will populate cache/block_b/... with .npy feature files)!python stg-real-eval/src/run_eval_real.py --config stg-real-eval/configs/block_b_cmp.yamlprint("Cache preview:")!find cache/block_b -maxdepth 3 -type f | head -n 20

## 8) Optional sanity test on Cityscapes demo subset
This cell runs the subset selector to copy a few frames from the `demoVideo` split and tests the pipeline.

In [ ]:
#@title Create and test a Cityscapes demo subset
%cd $ROOT

CITYSCAPES_DEMO = CITYSCAPES_SEQ_ROOT
DEST_SUBSET = "/content/stg-real-eval/datasets/cityscapes_subset"

!python stg-real-eval/src/scripts/select_cityscapes_subset.py --root "$CITYSCAPES_DEMO" --num_towns 1 --num_frames 10 --dest "$DEST_SUBSET"

# Patch config to use the subset as seq_root
import yaml
from pathlib import Path
cfg_path = Path(ROOT) / "stg-real-eval/configs/block_b_cityscapes.yaml"
cfg = yaml.safe_load(cfg_path.read_text())
cfg["paths"]["seq_root"] = DEST_SUBSET
cfg_path.write_text(yaml.safe_dump(cfg))

print("Running Block B pipeline on the subset...")
!python stg-real-eval/src/run_eval_real.py --config stg-real-eval/configs/block_b_cityscapes.yaml

# Visualize subset contents (first 3 frames)
from IPython.display import Image, display
subset_imgs = sorted(list(Path(DEST_SUBSET).glob("**/*.png")))[:3]
for im in subset_imgs:
    display(Image(filename=str(im)))
print(f"Displayed {len(subset_imgs)} images from subset.")


In [ ]:
#@title Cache sanity checkimport osprint("Torch Hub home:", os.environ.get("TORCH_HOME", "(default)"))!du -sh cache/block_b || echo "No cache yet"!find cache/block_b -maxdepth 2 -type d -print